In [1]:
clean_up=True
%run StdPackages.ipynb

# Emission data for 1990-2020

Load data:
* MRU1 provides an account of emissions based on the 69 sector grouping that the rest of the IO data is defined over. It is split into a couple of emission types that needs to be adjusted to arrive at a simple total.
* MRO2 is an aggregate account of emissions. This is used to scale the level of emissions from the 69 sector definition (emissions from Danish economic activities) proportionally to a level that is consistent with the UNFCC method that the climate targets are formulated in.

In [2]:
rawData = read.simpleLoad(os.path.join(d['data'],'Emissions.xlsx'))
MRU1 = pd.DataFrame(rawData['MRU1'].values)
MRO2 = pd.DataFrame(rawData['MRO2'].values)
namesVariablesMRU1 = {'Carbon dioxide (CO2) excl. from biomass, (1000 tonnes)': 'CO2',
                      'Nitrous oxide (N2O), (tonnes)': 'N2O',
                      'Methane (CH4), (tonnes)': 'CH4',
                      'Sulphur hexafluoride (SF6), (tonnes CO2-equivalents)': 'SF6',
                      'Perfluorocarbons (PFC), (tonnes CO2-equivalents)': 'PFC',
                      'Hydrofluorocarbons (HFC), (tonnes CO2-equivalents)': 'HFC'
                     }
CO2Factors = pd.Series([1, 298/1000, 25/1000, 1/1000, 1/1000, 1/1000], index = pd.Index(['CO2','N2O','CH4','SF6','PFC','HFC'], name = 'EmissionType'))

Use MRO2 to define scale of emissions from Danish UNFCCC accounting:

In [3]:
x = MRO2.iloc[2,2:]
x.index = pd.Index(MRO2.iloc[0,2:], name='t')

Assign indices:
* The first column identifies the variable. The category data is sparse in the sense that column contains ```None``` types for every cell after the first row that indicates a new variable.
* The second column identifies the sector index. Here we simply strip the strings from the definition and only use codes.

*Index 1 - variable name:*

In [4]:
variableRows = MRU1[0].dropna()
for i in range(len(variableRows)):
    if i < len(variableRows)-1:
        MRU1.iloc[variableRows.index[i]:variableRows.index[i+1], 0] = namesVariablesMRU1[variableRows.iloc[i]]
    else:
        MRU1.iloc[variableRows.index[i]:, 0] = namesVariablesMRU1[variableRows.iloc[i]]

*Index 2 - sector:*

In [5]:
branchIndex = MRU1[1].dropna().str.split(' ').str[0]
MRU1.iloc[branchIndex.index[0]:branchIndex.index[-1]+1,1] = branchIndex

Set relevant rows/columns as indices - remove '..' with zeros:

In [6]:
df = MRU1.iloc[1:,].set_index([0,1]).rename_axis(['EmissionType','s'])
df.columns = pd.Index(MRU1.iloc[0,2:], name = 't')
df = df.replace("..",0)

Aggregate emission types to CO2e:

In [7]:
M0 = pyDatabases.pdSum(df.stack()*CO2Factors, 'EmissionType')

Scale emissions:

In [8]:
M = M0 * x/ pyDatabases.pdSum(M0,'s')